In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
test_df = pd.read_csv("./input/test.csv")   # test set
train_df = pd.read_csv("./input/train.csv")
#sample_submission_df = pd.read_csv("./input/sample_submission.csv")

## 귀찮으시면, 
# ctrl+f  + 끝 
## 부터 보시면 러닝 돌린거부터 볼 수 있습니다~

## 아래부터 전처리 갑니다.

In [52]:
train_df.keys()

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [53]:
train_df.shape

(26729, 10)

## 전처리

성별 column과 중성화 column 추가

In [54]:
# functions to get new parameters from the column
def get_sex(x):
    x = str(x)
    if x.find('Male') >= 0: return 'male'
    if x.find('Female') >= 0: return 'female'
    return 'unknown'
def get_neutered(x):
    x = str(x)
    if x.find('Spayed') >= 0: return 'neutered'
    if x.find('Neutered') >= 0: return 'neutered'
    if x.find('Intact') >= 0: return 'intact'
    return 'unknown'

In [55]:
train_df['Sex'] = train_df.SexuponOutcome.apply(get_sex)
train_df['Neutered'] = train_df.SexuponOutcome.apply(get_neutered)

In [56]:
# 이름과 전처리
train_df['Name'] = train_df['Name'].replace(np.nan, 'Unknown', regex=True)
train_df['AgeuponOutcome'] = train_df['AgeuponOutcome'].replace(np.nan, '0 days', regex=True)

In [57]:
from pandas import Series

bydays = []

for i in train_df['AgeuponOutcome']:
    if i.find(' ') >= 0:
        f, s = i.split(' ')
        if s == 'year' or s == 'years':
            f = int(f) * 365
            bydays.append(f)
        elif s == 'month' or s == 'months':
            f = int(f) * 30
            bydays.append(f)
        elif s == 'week' or s == 'weeks':
            f = int(f) * 7
            bydays.append(f)
        elif s == 'day' or s == 'days':
            f = int(f)
            bydays.append(f)
    else:
        bydays.append(i)

train_df['AgeuponOutcome'] = Series(bydays)

In [58]:
# Name 유무 변수 생성

def is_name(x):
    x = str(x)
    if x.find('Unknow') >= 0: return 0
    else: return 1

train_df['isName'] = train_df.Name.apply(is_name)

In [59]:
# number of color 색상의 갯수, 0은 한색, 1은 두색이상
def get_num(x):
    x = str(x)
    if x.find('/') >= 0: return 1
    else: return 0

train_df['numCol'] = train_df.Color.apply(get_num)

In [60]:
train_df.tail()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Sex,Neutered,isName,numCol
26724,A702446,Unknown,2015-05-14 11:56:00,Transfer,Partner,Cat,Intact Male,30,Domestic Shorthair Mix,Brown Tabby/White,male,intact,0,1
26725,A718934,Unknown,2016-01-20 18:59:00,Transfer,SCRP,Cat,Spayed Female,90,Domestic Shorthair Mix,Brown Tabby,female,neutered,0,0
26726,A698128,Zeus,2015-03-09 13:33:00,Adoption,NaN,Dog,Neutered Male,1460,Old English Bulldog Mix,White/Tan,male,neutered,1,1
26727,A677478,Unknown,2014-04-27 12:22:00,Transfer,Partner,Cat,Intact Male,28,Domestic Shorthair Mix,Black,male,intact,0,0
26728,A706629,Unknown,2015-07-02 09:00:00,Transfer,SCRP,Cat,Intact Male,365,Domestic Shorthair Mix,Brown Tabby/White,male,intact,0,1


In [61]:
# Color 단순화 1단계 '/' 풀기
from pandas import Series

bymain = []

for i in train_df['Color']:
    i = str(i)
    if i.find('/') >= 0:
        f, s = i.split('/')
        bymain.append(f)
    else:
        bymain.append(i)

train_df['newColor'] = Series(bymain)

In [62]:
train_df['newColor'].value_counts()

Black                6422
White                3344
Brown Tabby          2592
Brown                1951
Tan                  1674
Orange Tabby         1299
Blue                 1199
Tricolor              800
Red                   779
Brown Brindle         699
Blue Tabby            678
Tortie                580
Calico                552
Chocolate             448
Torbie                398
Sable                 324
Cream Tabby           277
Buff                  267
Yellow                230
Gray                  227
Cream                 221
Fawn                  209
Lynx Point            183
Blue Merle            165
Seal Point            158
Black Brindle          99
Flame Point            85
Gold                   77
Brown Merle            72
Black Smoke            62
Black Tabby            61
Silver                 53
Red Merle              52
Gray Tabby             51
Blue Tick              44
Orange                 43
Silver Tabby           41
Red Tick               40
Lilac Point 

In [63]:
def get_mainC(x):
    x = str(x)
    if x.find('Black') >= 0: return 'dark'
    if x.find('White') >= 0: return 'light'
    if x.find('Brown') >= 0: return 'dark'
    if x.find('Orange') >= 0: return 'light'
    if x.find('Blue') >= 0: return 'medium'
    if x.find('Red') >= 0: return 'medium'
    
    if x.find('Tan') >= 0: return 'medium'
    if x.find('Tortie') >= 0: return 'dark'
    if x.find('Calico') >= 0: return 'medium'
    if x.find('Torbie') >= 0: return 'medium'
    if x.find('Buff') >= 0: return 'light'
    if x.find('Sable') >= 0: return 'dark'
    if x.find('Cream') >= 0: return 'light'
    if x.find('Yellow') >= 0: return 'light'
    if x.find('Lynx') >= 0: return 'light'
    if x.find('Chocolate') >= 0: return 'dark'
    if x.find('Gray') >= 0: return 'dark'
    if x.find('Liver') >= 0: return 'dark'
    if x.find('Flame') >= 0: return 'light'
    if x.find('Agouti') >= 0: return 'dark'
    if x.find('Pink') >= 0: return 'light'
    if x.find('Ruddy') >= 0: return 'medium'
    if x.find('Gold') >= 0: return 'light'
    if x.find('Silver') >= 0: return 'light'
    if x.find('Lilac') >= 0: return 'light'
    if x.find('Seal') >= 0: return 'medium'
    if x.find('Fawn') >= 0: return 'light'
    if x.find('Apricot') >= 0: return 'light'
    else: return x

train_df['newColor2'] = train_df.newColor.apply(get_mainC)

In [64]:
train_df['newColor2'].value_counts()

dark        13670
light        6429
medium       5830
Tricolor      800
Name: newColor2, dtype: int64

In [65]:
train_df.keys()

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color',
       'Sex', 'Neutered', 'isName', 'numCol', 'newColor', 'newColor2'],
      dtype='object')

In [66]:
bydate = []
bytime = []

for i in train_df['DateTime']:
    if i.find(' ') >= 0:
        f, s = i.split(' ')
        bydate.append(f)
        bytime.append(s)
    else:
        pass

train_df['date'] = Series(bydate)
train_df['time'] = Series(bytime)

In [67]:
byyear = []
bymonth = []
byday = []

for i in train_df['date']:
    if i.find('-') >= 0:
        f, s, d = i.split('-')
        byyear.append(f)
        bymonth.append(s)
        byday.append(d)
    else:
        pass

train_df['year'] = Series(byyear)
train_df['month'] = Series(bymonth)
train_df['day'] = Series(byday)

In [68]:
byhour = []
byminu = []
bysec = []

for i in train_df['time']:
    if i.find(':') >= 0:
        f, s, d = i.split(':')
        byhour.append(f)
        byminu.append(s)
        bysec.append(d)
    else:
        pass

train_df['hour'] = Series(byhour)
train_df['minute'] = Series(byminu)
train_df['sec'] = Series(bysec)

In [69]:
# useing variables
uvar = ['OutcomeType', 'AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName', 'numCol', 'newColor', 'year', 'month', 'day', 'hour', 'minute', 'sec']
uvar_c = ['AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName', 'numCol', 'newColor', 'year', 'month', 'day', 'hour', 'minute', 'sec']

In [70]:
train_df[uvar].keys()

Index(['OutcomeType', 'AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome',
       'isName', 'numCol', 'newColor', 'year', 'month', 'day', 'hour',
       'minute', 'sec'],
      dtype='object')

In [71]:
df_uv = train_df[uvar]
df_uv.target = df_uv.iloc[:,0]
df_uv.data = df_uv.iloc[:,1:]

In [81]:
# df_uv.data   더미화

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

#df_uv.data['OutcomeSubtype'] = encoder.fit_transform(df_uv.data['OutcomeSubtype'])

df_uv.data['AnimalType'] = encoder.fit_transform(df_uv.data['AnimalType'])
df_uv.data['Sex'] = encoder.fit_transform(df_uv.data['Sex'])
df_uv.data['Neutered'] = encoder.fit_transform(df_uv.data['Neutered'])
df_uv.data['numCol'] = encoder.fit_transform(df_uv.data['numCol'])
df_uv.data['newColor'] = encoder.fit_transform(df_uv.data['newColor'])
#df_uv.data['AgeGroup'] = encoder.fit_transform(df_uv.data['AgeGroup'])

#df_uv.data['newColor2'] = encoder.fit_transform(df_uv.data['newColor2'])

In [73]:
df_uv.data.keys()

Index(['AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName', 'numCol',
       'newColor', 'year', 'month', 'day', 'hour', 'minute', 'sec'],
      dtype='object')

In [74]:
df_uv.target.keys()

RangeIndex(start=0, stop=26729, step=1)

In [75]:
df_uv.target.value_counts()

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: OutcomeType, dtype: int64

In [76]:
dog = df_uv[df_uv['AnimalType'] == 'Dog']
cat = df_uv[df_uv['AnimalType'] == 'Cat']

dog.target = dog.iloc[:,0]
dog.data = dog.iloc[:,1:]

cat.target = cat.iloc[:,0]
cat.data = cat.iloc[:,1:]

In [77]:
# 더미화

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

#df_uv.data['OutcomeSubtype'] = encoder.fit_transform(df_uv.data['OutcomeSubtype'])

dog.data['AnimalType'] = encoder.fit_transform(dog.data['AnimalType'])
dog.data['Sex'] = encoder.fit_transform(dog.data['Sex'])
dog.data['Neutered'] = encoder.fit_transform(dog.data['Neutered'])
dog.data['numCol'] = encoder.fit_transform(dog.data['numCol'])
dog.data['newColor'] = encoder.fit_transform(dog.data['newColor'])
#dog.data['AgeGroup'] = encoder.fit_transform(dog.data['AgeGroup'])

cat.data['AnimalType'] = encoder.fit_transform(cat.data['AnimalType'])
cat.data['Sex'] = encoder.fit_transform(cat.data['Sex'])
cat.data['Neutered'] = encoder.fit_transform(cat.data['Neutered'])
cat.data['numCol'] = encoder.fit_transform(cat.data['numCol'])
cat.data['newColor'] = encoder.fit_transform(cat.data['newColor'])
#cat.data['AgeGroup'] = encoder.fit_transform(cat.data['AgeGroup'])

#df_uv.data['newColor2'] = encoder.fit_transform(df_uv.data['newColor2'])

In [ ]:
train_df[train_df['AnimalType'] == 'Cat'][train_df['Breed']!='Domestic Shorthair Mix'][train_df['Breed']!='Domestic Longhair Mix'][train_df['Breed']!='Domestic Medium Hair Mix'][train_df['Breed']!='Siamese Mix']

# 전처리 끝

### df_uv.target & df_uv.data 
### dog.target & .data
### cat.target & .data

- df_uv.target  ->  ['OutcomeType']
- df_uv.data  ->  ['AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName', 'newColor2']

- .target -> ['OutcomeType']
- .data -> ['AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)

## DecisionTree DOG

In [ ]:
clf = clf.fit(dog.data, dog.target)

In [ ]:
clf.score(dog.data, dog.target)

In [ ]:
from IPython.display import Image
from sklearn import tree
import pydotplus 
import graphviz

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,
                        feature_names=uvar_c,
                        class_names=train_df.OutcomeType,
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

## DecisionTree CAT

In [ ]:
clf = clf.fit(cat.data, cat.target)

In [ ]:
clf.score(cat.data, cat.target)

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,
                        feature_names=uvar_c,
                        class_names=train_df.OutcomeType,
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

In [ ]:
# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bagging = BaggingClassifier(DecisionTreeClassifier(), max_samples=0.5, max_features=0.5)

## Bagging DOG

In [ ]:
bagging.fit(dog.data, dog.target)

In [ ]:
bagging.score(dog.data, dog.target)

## Bagging Cat

In [ ]:
bagging.fit(cat.data, cat.target)

In [ ]:
bagging.score(cat.data, cat.target)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
bagging = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)

In [ ]:
bagging.fit(dog.data, dog.target)

In [ ]:
bagging.score(dog.data, dog.target)

In [ ]:
bagging.fit(cat.data, cat.target)

In [ ]:
bagging.score(cat.data, cat.target)

In [78]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier

In [79]:
rfc = RandomForestClassifier(n_estimators=10)

In [ ]:
rfc.fit(dog.data, dog.target)

In [ ]:
rfc.score(dog.data, dog.target)

In [ ]:
rfc.fit(cat.data, cat.target)

In [ ]:
rfc.score(cat.data, cat.target)

In [ ]:
rfc.predict_proba(cat.data)

In [82]:
rfc.fit(df_uv.data, df_uv.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [83]:
rfc.score(df_uv.data, df_uv.target)

0.98529686856971832

In [96]:
# feature importance
rfc.feature_importances_

array([ 0.02581334,  0.03505621,  0.10562026,  0.18036435,  0.04410929,
        0.01743239,  0.07710993,  0.03868988,  0.0892565 ,  0.12337045,
        0.11297837,  0.15019903,  0.        ])

In [97]:
pd.Series(rfc.feature_importances_, index=uvar_c)

AnimalType        0.025813
Sex               0.035056
Neutered          0.105620
AgeuponOutcome    0.180364
isName            0.044109
numCol            0.017432
newColor          0.077110
year              0.038690
month             0.089256
day               0.123370
hour              0.112978
minute            0.150199
sec               0.000000
dtype: float64

In [ ]:
# Boosting

In [ ]:
# target 더미화
dog.target = encoder.fit_transform(dog.target)
cat.target = encoder.fit_transform(cat.target)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),
                          n_estimators=300, random_state=1)

In [ ]:
regr_1.fit(dog.data, dog.target)
regr_2.fit(dog.data, dog.target)

In [ ]:
print(regr_1.score(dog.data, dog.target))
print(regr_2.score(dog.data, dog.target))

In [ ]:
regr_1.fit(cat.data, cat.target)
regr_2.fit(cat.data, cat.target)

In [ ]:
print(regr_1.score(cat.data, cat.target))
print(regr_2.score(cat.data, cat.target))

## boosting 잘 한건 지 모르겟음.

## 시간 오래걸림... 주의 요함... (근데 성적은 고만고만...)

In [ ]:
# SVM

In [ ]:
from sklearn import svm
svmex = svm.SVC()
svmex.fit(dog.data, dog.target)

In [ ]:
svmex.score(dog.data, dog.target)

In [ ]:
svmex.fit(cat.data, cat.target)

In [ ]:
svmex.score(cat.data, cat.target)

In [89]:
test_df = pd.read_csv("./input/test.csv")

In [90]:
test_df['Sex'] = test_df.SexuponOutcome.apply(get_sex)
test_df['Neutered'] = test_df.SexuponOutcome.apply(get_neutered)

test_df['Name'] = test_df['Name'].replace(np.nan, 'Unknown', regex=True)
test_df['AgeuponOutcome'] = test_df['AgeuponOutcome'].replace(np.nan, '0 days', regex=True)

bydays = []
for i in test_df['AgeuponOutcome']:
    if i.find(' ') >= 0:
        f, s = i.split(' ')
        if s == 'year' or s == 'years':
            f = int(f) * 365
            bydays.append(f)
        elif s == 'month' or s == 'months':
            f = int(f) * 30
            bydays.append(f)
        elif s == 'week' or s == 'weeks':
            f = int(f) * 7
            bydays.append(f)
        elif s == 'day' or s == 'days':
            f = int(f)
            bydays.append(f)
    else:
        bydays.append(i)
test_df['AgeuponOutcome'] = Series(bydays)

test_df['isName'] = test_df.Name.apply(is_name)

test_df['numCol'] = test_df.Color.apply(get_num)

bymain = []
for i in test_df['Color']:
    i = str(i)
    if i.find('/') >= 0:
        f, s = i.split('/')
        bymain.append(f)
    else:
        bymain.append(i)
test_df['newColor'] = Series(bymain)

test_df['newColor2'] = test_df.newColor.apply(get_mainC)

bydate = []
bytime = []
for i in test_df['DateTime']:
    if i.find(' ') >= 0:
        f, s = i.split(' ')
        bydate.append(f)
        bytime.append(s)
    else:
        pass
test_df['date'] = Series(bydate)
test_df['time'] = Series(bytime)

byyear = []
bymonth = []
byday = []
for i in test_df['date']:
    if i.find('-') >= 0:
        f, s, d = i.split('-')
        byyear.append(f)
        bymonth.append(s)
        byday.append(d)
    else:
        pass
test_df['year'] = Series(byyear)
test_df['month'] = Series(bymonth)
test_df['day'] = Series(byday)

byhour = []
byminu = []
bysec = []
for i in test_df['time']:
    if i.find(':') >= 0:
        f, s, d = i.split(':')
        byhour.append(f)
        byminu.append(s)
        bysec.append(d)
    else:
        pass
test_df['hour'] = Series(byhour)
test_df['minute'] = Series(byminu)
test_df['sec'] = Series(bysec)

tvar = ['AnimalType', 'Sex', 'Neutered', 'AgeuponOutcome', 'isName', 'numCol', 'newColor', 'year', 'month', 'day', 'hour', 'minute', 'sec']


df_tvar = test_df[tvar]

df_tvar['Sex'] = encoder.fit_transform(df_tvar['Sex'])
df_tvar['Neutered'] = encoder.fit_transform(df_tvar['Neutered'])
df_tvar['numCol'] = encoder.fit_transform(df_tvar['numCol'])
df_tvar['newColor'] = encoder.fit_transform(df_tvar['newColor'])

C:\Python36\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python36\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python36\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python36\lib\sit

In [93]:
df_tvar['AnimalType'] = encoder.fit_transform(df_tvar['AnimalType'])

C:\Python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
rfc.predict_proba(df_tvar)[1][0]

0.5

In [100]:
Predicted = rfc.predict_proba(df_tvar)

In [121]:
import csv
    
with open('answer.csv', 'w') as csvfile:
    #fieldnames = ['ID', 'Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer']
    #writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer = csv.writer(csvfile)
    writer.writerow(['ID','Adoption','Died','Euthanasia','Return_to_owner','Transfer'])
    for i in range(len(test_df.ID)): 
        writer.writerow([test_df.ID[i], Predicted[i][0], Predicted[i][1], Predicted[i][2],Predicted[i][3],Predicted[i][4]])